랭체인에는 5가지 정도 종류의 메모리가 있다.
각자 저장 방식도 다르고 각자만의 장단점이 있다.

챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없다.

오픈AI에서 제공하는 기본 API는 랭체인 없이 사용할 수 있는데 메모리를 지원하지 않는다.(stateless)  
즉, 모델에게 어떤 말을 건네면 모델은 답을 한 직후에 모든 대화 내용을 까먹게 된다.(저장 X)

챗GPT에는 메모리가 탑재되어 있기 때문에 대화하는 느낌을 들게 한다.


### 첫 번째 메모리는 Conversation Buffer Memory 라고 한다.

이 메모리는 단순하다. 그냥 이전 대화 내용 전체를 저장하는 메모리이다.   
이 메모리의 단점은 대화 내용이 길어질수록 메모리도 커지니까 비효율적이라는 것.  



In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context(
    {"input": "Hi"},
    {"output": "how are  you?"},
)
memory.load_memory_variables({})

In [ ]:
memory.save_context(
    {"input": "Hi22"},
    {"output": "how are  you?222"},
)
memory.load_memory_variables({})

이 메모리는 text completion 할때 유용하다. 
- 예측을 해야할 때.
- 텍스트를 자동완성하고 싶을 때.

그러나 만약 챗모델과 작업을 하면 AI메세지와 Human 메세지가 다 필요하다.

원한다면 다음과 같이 작성할 수 있다.

```python
memory = ConversationBufferMemory(return_messages=True)
```

알아둬야 하는 건 메모리 종류와 무관하게 API들은 다 똑같다는 점이다!  
모든 메모리는 save_context, load_memory_variables 라는 함수를 갖고 있다.  



In [ ]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context(
    {"input": "Hi"},
    {"output": "how are  you?"},
)
memory.load_memory_variables({})

In [ ]:
memory.save_context(
    {"input": "Hi"},
    {"output": "how are  you?"},
)
memory.load_memory_variables({})

In [ ]:
memory.save_context(
    {"input": "Hi"},
    {"output": "how are  you?"},
)
memory.load_memory_variables({})

In [ ]:
memory.save_context(
    {"input": "Hi"},
    {"output": "how are  you?"},
)
memory.load_memory_variables({})

history 를 보면 HumanMessage랑 AIMessage로 바뀌었다.

---

앞으로 배워볼 다른 메모리를도 동일한 API를 갖고 있다는 사실을 알아두자.

우선 메모리를 만들고, 챗모델을 위한건지 아닌지 선택하고,  

챗 모델을 위한 게 아니라면 False 또는 아예 return_message를 빼도 된다.  

그러면 history가 문자열로 표시될 것이다.

챗모델을 사용하고 싶다면? return_message=True 로 설정하면 된다.  

그러면 챗모델이 사용할 수 있는 형태로 출력이 된다.

API 는 메모리 종류와 상관없이 다 똑같다.

save_context - input, output 다 똑같고,

대화 기록을 불러오기 위해선 load_memory_variables 라는 함수를 실행.



### 두 번째는 Conversation Buffer Window Memory 에 대해 알아보자.

ConversationBufferMemory 와 다르게 ConversationBufferWindowMemory 는  
대화의 특정 부분만을 저장하는 메모리다.

예를 들어 제일 최근 5개의 메세지만 저장한다고 하면  
6번째 메세지가 추가 됐을 때 가장 오래된 메세지는 버려지는 방식이다.

메모리를 특정 크기로 유지할 수 있다는 게 이 ConversationBufferWindowMemory의 큰 장점이다.  
단점은 챗봇이 전체가 아닌 최근 대화에만 집중한다는 것.



In [11]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    # k =  버퍼 윈도우의 사이즈: 몇 개의 메세지를 저장할지
    k=4,
)

메세지를 추가 하기 쉽게 함수를 만들어보자.

In [4]:
def add_message(input, output):
    memory.save_context(
        {"input": input},
        {"output": output},
    )

이제 메세지를 추가해보자.

In [13]:
add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

In [ ]:
memory.load_memory_variables({})

In [ ]:
add_message(5, 5)
memory.load_memory_variables({})

### 세 번째는 ConversationSummaryMemory

ConversationSummaryMemory 는 llm을 사용한다.


즉, 메모리를 실행하는 데 비용이 든다는 뜻.

In [1]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryMemory(llm=chat)

ConversationSummaryMemory는 message 그대로 저장하는 것이 아니라   
conversation의 요약을 자체적으로 해주는 것이다.

초반에는 ConversationSummaryMemory는 이전보다 더 많은 토큰과 저장공간을 차지하게 될거다.  
하지만 conversation 버퍼 메모리를 사용하고 있어서 대화가 진행될 수록 저장된 모든 메세지가 매우 많아지면서 잘 연결될 것이다. 

conversation 메세지가 많아질수록 ConversationSummaryMemory 의 도움을 받아 요약하는 것이 토큰의 양도 줄어들면서 훨씬 나은 방법이 된다.

In [2]:
def get_history():
    return memory.load_memory_variables({})

In [5]:
add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()

{'history': "The human introduces themselves as Nicolas from South Korea. The AI responds by expressing admiration for Nicolas's location."}

In [6]:
add_message("South korea is so beautiful", "I wish I could go!!!")
get_history()

{'history': "The human introduces themselves as Nicolas from South Korea. The AI responds by expressing admiration for Nicolas's location and wishes it could go there."}

In [7]:
add_message("South korea is so cold today", "I wish It turns warm!!!")
get_history()

{'history': "The human introduces themselves as Nicolas from South Korea. The AI responds by expressing admiration for Nicolas's location and wishes it could go there. Nicolas mentions that South Korea is cold today, and the AI expresses a desire for it to turn warm."}

In [8]:
get_history()

{'history': "The human introduces themselves as Nicolas from South Korea. The AI responds by expressing admiration for Nicolas's location and wishes it could go there. Nicolas mentions that South Korea is cold today, and the AI expresses a desire for it to turn warm."}

### 네 번째는 ConversationSummaryBufferMemory

Conversation Summary Memory, Conversation Buffer Memory 의 결합이다.  
이것이 하는 일은, 메모리에 보내온 메세지의 수를 저장하는 것이다.

또한 우리가 limit에 다다른 순간, 그냥 무슨 일이 일어났는지 잊어버리는 것 대신  
오래된 메세지들을 summarize(요약) 할 것이다.  
즉, 가장 최근의 상호작용을 계속 추적한다는 것.  

주고받은 메세지 모두 잊혀지지 않고 요약됨.


max_token_limit - 이것은 가능한 메세지 토큰 수의 최대값을 의미한다. 메세지들이 요약되기 전에.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=150,
)


def add_message(input, output):
    memory.save_context(
        {"input": input},
        {"output": output},
    )


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()

# add_message("South korea is so beautiful", "I wish I could go!!!")

# get_history()

get_history를 실행해 보면 우리는 메세지를 그대로 저장하고 있다.
```python
{'history': "Human: Hi I'm Nicolas, I live in South Korea\nAI: Wow that is so cool!"}
```
return_messages 를 True로 지정해보자.

In [3]:
memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=80,
    return_messages=True,
)

return_messages 를 True로 설정하는 이유는 이것을 채팅 모델에 사용할 것이기 때문이다.

In [ ]:
add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()

질문을 하나 더 해보고 다시 history를 확인해보자.

In [ ]:
add_message("South korea is so beautiful", "I wish I could go!!!")

get_history()

In [6]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
get_history()

In [ ]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
get_history()

위에서 보는것처럼 limit에 도달하면 오래된 메세지들이 요약되어서 보인다.

---


### 다섯번 째는 Conversation Knowledget Graph Memory 이다.

이것도 llm 사용.  
이건 대화 중의 엔티티의 knowledget graph를 만든다.  

가장 중요한 것들만 뽑아내는 요약본 같은 것.

knowledge graph에서 히스토리를 가지고 오지 않고 엔티티를 가지고 오기 때문에 get_history 는 지웠다.  

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi, I'm Yoon, I live in South Korea", "Wow that is so awesome!")

"Hi, I'm Yoon, I live in South Korea" 라는 메세지를 보내고, Memory를 불러와보자.

In [ ]:
memory.load_memory_variables({"input": "who is yoon"})

요약하자면 대화에서 entity를 뽑아내는 것이라 할 수 있다.


In [4]:
add_message("yoon likes money", "wow that is true")

In [ ]:
memory.load_memory_variables({"input": "what deos yoon like?"})

이렇게 다 수동으로 정보를 입력하면 챗봇이 이걸 어떻게 활용하게 될까?

뒤에 memory를 사용하는 다른 chain들을 살펴보고 이걸 활용할 수 있는 쉬운 방법을 알아보자.    
계속 .save_context 하고 .load_memory_variables를 계속 하는 건 번거롭다.  
뒤에 자동으로 즉시 메모리를 활용할 수 있는 방법이 있다.  
그럼 load_memory_variables를 할 필요도 없고 save_context를 할 필요도 없다.